# Credit Risk Project

First and foremost the goal is to learn about credit risk and applying machine learning and analysis.

We need a data set to work with. I am going to look on Kaggle.

Before starting, I am going to lay out some stuff I know about credit risk and how machine learning applies. First credit risk is quantifying or decidimg how risky it is to loan out money to a person or organization based on the information on their application. Some ML algorithms that would be useful would be supervised learning to help the algorithm make decisions on whether to give a loan to someone based on their labels of past successful or unsuccessful algorithms. ML algorithms that work well with tabular data like XGBoost or Random Forest would be good for this situation. Unsupervised algorithms would be use in order to group applications into certain categories without specifically specifying their labels. They would be group on how similar they are to other. Classification algorithms are the way to go on this one (I think... I haven't seen the data). Specifically Logistical Classification helps with probability of a grant or maybe even supper vector machines that could split the accepted vs unaccepted applications. Knowing all this so far let's do some EDA and figure it out from there. 

In [ ]:
import kagglehub

# Download latest versionc:\Users\ty725\.cache\kagglehub\datasets\wordsforthewise\lending-club\versions\3
path = kagglehub.dataset_download("wordsforthewise/lending-club")

print("Path to dataset files:", path)

In [1]:
path = "C:\\Users\\ty725\.cache\kagglehub\\datasets\wordsforthewise\\lending-club\\versions\\3"
path1 = path + "\\accepted_2007_to_2018q4.csv\\accepted_2007_to_2018Q4.csv"
path2 = path + "\\rejected_2007_to_2018q4.csv\\rejected_2007_to_2018Q4.csv"

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\ty725\AppData\Local\Temp\ipykernel_15960\25542923.py:1: SyntaxWarning: invalid escape sequence '\.'
  path = "C:\\Users\\ty725\.cache\kagglehub\\datasets\wordsforthewise\\lending-club\\versions\\3"


Okay these datasets are huge! I think this is a good oppurtunity to use PySpark. I heard that it works really well with huge datasets. 

## Dataset EDA

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, isnull, sum, lit

# Start new session
spark = SparkSession.builder \
    .appName("NewSession") \
    .getOrCreate()

In [3]:
dfa = spark.read.csv(path1, header=True)

Ok so there's a one thing to notice other than the amount of rows being huge: the columns are very different for the rejected pile vs the accepted pile. The accepted pile includes way more information about the actual loan that give out as well. So before we start cleaning, we have to figure which of the most important features/columns we need to clean. The best way to do that is to figure out which of the columns match because the goal is to get the accepted and the reject in one pile for the ML algo. 

After some inspiration, I understand how I am supposed to approach this problem. Focus on only accepted loans to predict the probability of default among accepted loans. The key column here is the loan status, mainly between statuses "Charged Off" vs "Fully Paid".

Ok so the next step is to load it on to the cloud and query it so we can get a dirty dataset. We'll use PySpark to clean it on the occasion that the dataset is still huge.

In [4]:
dfa.createOrReplaceTempView('loans')

In [5]:
q = spark.sql('''
SELECT loan_status, loan_amnt, total_pymnt, total_rec_prncp, recoveries
FROM loans
WHERE id = '100006980'
''')
q.show()

+-----------+---------+----------------+---------------+----------+
|loan_status|loan_amnt|     total_pymnt|total_rec_prncp|recoveries|
+-----------+---------+----------------+---------------+----------+
| Fully Paid|  40000.0|44943.5462511188|        40000.0|       0.0|
+-----------+---------+----------------+---------------+----------+



In [6]:
df = spark.sql('''
    SELECT * FROM loans
    WHERE loan_status = 'Charged Off' OR  loan_status = 'Fully Paid' 
               OR loan_status = 'Default'
''')
df.show()

+--------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+----------+--------------------+----+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+------------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-

In [7]:
df = df.withColumn(
    "loan_status",
    when(df.loan_status.isin("Charged Off", "Default"), "Charged Off")
    .otherwise("Fully Paid")
)

In [8]:
#df.describe().show()

In [9]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = false)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string

In [10]:
rows = df.count()
rows

1345349

In [11]:
def nullCount(dft):
    nullCounts = dft.agg(*[
        ((sum(isnull(col(c)).cast('integer')) / rows) * 100).alias(c)
        for c in dft.columns
    ])
    nullCounts.show()

nullCount(df)

+---+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+-----------------+------------------+--------------+----------+-------------------+-------+-----------+----------+---+-----------------+-------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+----------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+------------------+--------------------+-----------------+--------------------+--------------------+-------------------+--------------------------+---------------------------+--------------------+----------------

I know I keep saying this but this dataset really is huge. My first approach was to go through each of attributes, figure out what they mean, and then figure out if they are useful. The thing is there is 151 attributes and I don't have all night here. We can't have a lot of attributes because this can cause the model to overfit, the key is generalization for a good ML model. So instead of getting 151 attributes down to 20 or so, it's better to hand pick the key features we need for our model and then clean that.

In [259]:
features = '''
id, loan_status, loan_amnt, int_rate, installment, term, grade, sub_grade, emp_length,
home_ownership, annual_inc, verification_status, purpose, dti, delinq_2yrs, 
inq_last_6mths, fico_range_low, fico_range_high, total_rec_prncp, recoveries
'''

def cleanStrings(strings):
    strings = strings.split(',')
    for i, string in enumerate(strings):
        strings[i] = string.strip()
    return strings



In [13]:
features = cleanStrings(features)
df = df.select(features)

## Data Cleaning 

In [14]:
df.show(5)

+--------+-----------+---------+--------+-----------+----------+-----+---------+----------+--------------+----------+-------------------+------------------+-----+-----------+--------------+--------------+---------------+---------------+----------+
|      id|loan_status|loan_amnt|int_rate|installment|      term|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|           purpose|  dti|delinq_2yrs|inq_last_6mths|fico_range_low|fico_range_high|total_rec_prncp|recoveries|
+--------+-----------+---------+--------+-----------+----------+-----+---------+----------+--------------+----------+-------------------+------------------+-----+-----------+--------------+--------------+---------------+---------------+----------+
|68407277| Fully Paid|   3600.0|   13.99|     123.03| 36 months|    C|       C4| 10+ years|      MORTGAGE|   55000.0|       Not Verified|debt_consolidation| 5.91|        0.0|           1.0|         675.0|          679.0|         3600.0|       0.0|
|6835508

In [15]:
nullCount(df)

+---+-----------+---------+--------+-----------+----+-----+---------+------------------+--------------+----------+-------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| id|loan_status|loan_amnt|int_rate|installment|term|grade|sub_grade|        emp_length|home_ownership|annual_inc|verification_status|purpose|                 dti|         delinq_2yrs|      inq_last_6mths|      fico_range_low|     fico_range_high|     total_rec_prncp|          recoveries|
+---+-----------+---------+--------+-----------+----+-----+---------+------------------+--------------+----------+-------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|0.0|        0.0|      0.0|     0.0|        0.0| 0.0|  0.0|      0.0|5.8361064675411365|           0.0|       0.0|                

Ok all missing values seems to be under 6% so it safe to drop all the rows that have a missing values. This saves a massive headache. Now there is something to note. Looking at the data, I assumed that if they're missing employment length, then that must mean that they're unemployed. Using data wrangler, some people have job titles but no employment length, so the unemployment assumption is not always valid. This is kind of significant to note becase employment length has the highest null count of 6%, everything else is less significant. Any ways, we'll drop any rows with missing cells, this dataset is huge.

In [16]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- loan_status: string (nullable = false)
 |-- loan_amnt: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- term: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- fico_range_low: string (nullable = true)
 |-- fico_range_high: string (nullable = true)
 |-- total_rec_prncp: string (nullable = true)
 |-- recoveries: string (nullable = true)



In [17]:
colsFloats ='loan_amnt, int_rate, installment, annual_inc, dti, total_rec_prncp, recoveries'
colsInts = 'delinq_2yrs, fico_range_low, fico_range_high, inq_last_6mths'

colsFloats = cleanStrings(colsFloats)
colsInts = cleanStrings(colsInts)

for column in colsFloats:
    df = df.withColumn(column, col(column).cast('float'))

for column in colsInts:
    df = df.withColumn(column, col(column).cast('int'))

In [18]:
rows = df.count()
rows

1345349

In [19]:
df = df.na.drop()
nullCount(df)

+---+-----------+---------+--------+-----------+----+-----+---------+----------+--------------+----------+-------------------+-------+---+-----------+--------------+--------------+---------------+---------------+----------+
| id|loan_status|loan_amnt|int_rate|installment|term|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|purpose|dti|delinq_2yrs|inq_last_6mths|fico_range_low|fico_range_high|total_rec_prncp|recoveries|
+---+-----------+---------+--------+-----------+----+-----+---------+----------+--------------+----------+-------------------+-------+---+-----------+--------------+--------------+---------------+---------------+----------+
|0.0|        0.0|      0.0|     0.0|        0.0| 0.0|  0.0|      0.0|       0.0|           0.0|       0.0|                0.0|    0.0|0.0|        0.0|           0.0|           0.0|            0.0|            0.0|       0.0|
+---+-----------+---------+--------+-----------+----+-----+---------+----------+--------------+---------

In [20]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- loan_status: string (nullable = false)
 |-- loan_amnt: float (nullable = true)
 |-- int_rate: float (nullable = true)
 |-- installment: float (nullable = true)
 |-- term: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- fico_range_low: integer (nullable = true)
 |-- fico_range_high: integer (nullable = true)
 |-- total_rec_prncp: float (nullable = true)
 |-- recoveries: float (nullable = true)



Ok so we're going to try something. We're going to apply One Hot Encoder becase we have some attributes that cannot be easily converted clean numbers for the algo to understand. One Hot Encoder is good for categorical data, namely in this case: loan_status, grade, sub_grade, emp_length (maybe, there's still hope for this one), home_ownership, verification_status, and purpose.

In [21]:
df = df.filter(col('home_ownership') != 'ANY')

In [22]:
otherInfo = df.filter(col('home_ownership') == 'OTHER').groupBy('loan_status').count()
otherInfo

DataFrame[loan_status: string, count: bigint]

In [23]:
NoneInfo =  df.filter(col('home_ownership') == 'NONE').groupBy('loan_status').count()
NoneInfo

DataFrame[loan_status: string, count: bigint]

In [24]:
common_owners = ["RENT", "OWN", "MORTGAGE"]

df = df.withColumn(
    "home_ownership",
    when(col("home_ownership").isin(common_owners), col("home_ownership"))
    .otherwise("OTHER")
)

In [25]:
encodedCols = 'emp_length, home_ownership, verification_status, purpose'
encodedCols = cleanStrings(encodedCols)
encodedCols

['emp_length', 'home_ownership', 'verification_status', 'purpose']

In [26]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

stages = []
for c in encodedCols:
    indexer = StringIndexer(inputCol = c, outputCol = c + '_')
    encoder = OneHotEncoder(inputCol = c + '_', outputCol = c + '-', dropLast = True)
    stages += [indexer, encoder]

pipeline = Pipeline(stages=stages)
encoded_df = pipeline.fit(df).transform(df)

In [27]:
encoded_df.show(5)

+--------+-----------+---------+--------+-----------+----------+-----+---------+----------+--------------+----------+-------------------+------------------+-----+-----------+--------------+--------------+---------------+---------------+----------+-----------+--------------+---------------+---------------+--------------------+--------------------+--------+--------------+
|      id|loan_status|loan_amnt|int_rate|installment|      term|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|           purpose|  dti|delinq_2yrs|inq_last_6mths|fico_range_low|fico_range_high|total_rec_prncp|recoveries|emp_length_|   emp_length-|home_ownership_|home_ownership-|verification_status_|verification_status-|purpose_|      purpose-|
+--------+-----------+---------+--------+-----------+----------+-----+---------+----------+--------------+----------+-------------------+------------------+-----+-----------+--------------+--------------+---------------+---------------+----------+-------

In [28]:
empInfo = encoded_df.groupBy("emp_length").count()
empInfo

DataFrame[emp_length: string, count: bigint]

In [29]:
empLoanInfo = encoded_df.groupBy("emp_length", "loan_status").count()
empLoanInfo

DataFrame[emp_length: string, loan_status: string, count: bigint]

In [30]:
import pandas as pd
empLoanInfo = empLoanInfo.toPandas()

In [31]:
def pivotIt(dft, index):
    pivot = dft.pivot_table(index = index, columns = "loan_status", values = "count", aggfunc = "sum")
    pivot['total'] = pivot.loc[:, 'Charged Off'] + pivot.loc[:, 'Fully Paid']
    pivot['default_percentage'] = (pivot.loc[:, 'Charged Off'] / pivot['total']) * 100
    return pivot.sort_values(ascending = True, by = 'default_percentage')

In [32]:
pivotIt(empLoanInfo, "emp_length")

loan_status,Charged Off,Fully Paid,total,default_percentage
emp_length,,,,
10+ years,83047,359045,442092,18.785004
6 years,12139,50571,62710,19.357359
7 years,11615,47989,59604,19.486947
5 years,16485,67621,84106,19.600266
4 years,15894,64615,80509,19.741892
2 years,24110,97588,121698,19.811336
9 years,10135,40786,50921,19.903380
8 years,12101,48591,60692,19.938377
3 years,21483,86065,107548,19.975267


With emp_length, after some data analysis I found that the difference is between 10+ and <1 is 1.75% (with 10+ having a lower charged off percentage), but the range between all the other numbers (2-9) was 0.6% and they were pretty random in order. Based on my findings I think I should use One Hot Encoding because the linear progression at least for the middle numbers is none existent. For the remaining columns (grade and subgrade) needs to be encoded in ordinally which basically means that the order matters when encoding. 

In [33]:
grade_df = encoded_df.groupBy("grade", "loan_status").count().toPandas()
pivotIt(grade_df, "grade")

loan_status,Charged Off,Fully Paid,total,default_percentage
grade,,,,
A,12795,209530,222325,5.755088
B,47961,321888,369849,12.967725
C,78814,280035,358849,21.962998
D,56205,131782,187987,29.898344
E,33680,54570,88250,38.164306
F,13620,16764,30384,44.826224
G,4300,4381,8681,49.533464


In [34]:
subgrade_df = encoded_df.groupBy("sub_grade", "loan_status").count().toPandas()
pivotIt(subgrade_df, "sub_grade")

loan_status,Charged Off,Fully Paid,total,default_percentage
sub_grade,,,,
A1,1243,39975,41218,3.015673
A2,1532,33576,35108,4.363678
A3,1851,34059,35910,5.154553
A4,3238,46082,49320,6.565288
A5,4931,55838,60769,8.114335
B1,6696,60318,67014,9.991942
B2,7636,62017,69653,10.962916
B3,9712,67472,77184,12.582919
B4,11317,67136,78453,14.425197


In [35]:
term_df = encoded_df.groupBy("term", "loan_status").count().toPandas()
pivotIt(term_df, "term")

loan_status,Charged Off,Fully Paid,total,default_percentage
term,,,,
36 months,147245,806889,954134,15.432319
60 months,100130,212061,312191,32.073314


This confirms the decision for Ordinal Encoding.

In [36]:
grades = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
levels = [1,2,3,4,5]
submap = {}
k = 0
ran = len(grades) * len(levels)
for i in range(len(grades)):
    for j in range(len(levels)):
        submap[grades[i] + str(levels[j])] = k
        k += 1
        
gMap = {}
for i in range(len(grades)):
    gMap[grades[i]] = i

termap = {' 36 months' : 0, ' 60 months' : 1}
resMap = {'Charged Off' : 0, 'Fully Paid' : 1}

In [37]:
def ordinal(data, column, mapp):
    column1 = column + '-'
    data = data.withColumn(column1, lit(None))
    for value, key in enumerate(mapp):
        data = data.withColumn(column1, when(col(column) == lit(key), value).otherwise(col(column1)))
    return data

In [38]:
encoded_df = ordinal(encoded_df, 'sub_grade', submap)
encoded_df = ordinal(encoded_df, 'grade', gMap)
encoded_df = ordinal(encoded_df, 'term', termap)
encoded_df = ordinal(encoded_df, 'loan_status', resMap)

In [39]:
encoded_df.show(5)

+--------+-----------+---------+--------+-----------+----------+-----+---------+----------+--------------+----------+-------------------+------------------+-----+-----------+--------------+--------------+---------------+---------------+----------+-----------+--------------+---------------+---------------+--------------------+--------------------+--------+--------------+----------+------+-----+------------+
|      id|loan_status|loan_amnt|int_rate|installment|      term|grade|sub_grade|emp_length|home_ownership|annual_inc|verification_status|           purpose|  dti|delinq_2yrs|inq_last_6mths|fico_range_low|fico_range_high|total_rec_prncp|recoveries|emp_length_|   emp_length-|home_ownership_|home_ownership-|verification_status_|verification_status-|purpose_|      purpose-|sub_grade-|grade-|term-|loan_status-|
+--------+-----------+---------+--------+-----------+----------+-----+---------+----------+--------------+----------+-------------------+------------------+-----+-----------+------

In [40]:
col_names = '''id, loan_status-, loan_amnt, int_rate, installment, term-, grade-, sub_grade-, 
emp_length-, home_ownership-, annual_inc, verification_status-, purpose-, dti, 
delinq_2yrs, inq_last_6mths, fico_range_low, fico_range_high, total_rec_prncp, recoveries
'''
col_names = cleanStrings(col_names)
df = encoded_df.select(col_names)
df.show(5)

+--------+------------+---------+--------+-----------+-----+------+----------+--------------+---------------+----------+--------------------+--------------+-----+-----------+--------------+--------------+---------------+---------------+----------+
|      id|loan_status-|loan_amnt|int_rate|installment|term-|grade-|sub_grade-|   emp_length-|home_ownership-|annual_inc|verification_status-|      purpose-|  dti|delinq_2yrs|inq_last_6mths|fico_range_low|fico_range_high|total_rec_prncp|recoveries|
+--------+------------+---------+--------+-----------+-----+------+----------+--------------+---------------+----------+--------------------+--------------+-----+-----------+--------------+--------------+---------------+---------------+----------+
|68407277|           1|   3600.0|   13.99|     123.03|    0|     2|        13|(10,[0],[1.0])|  (3,[0],[1.0])|   55000.0|       (2,[1],[1.0])|(13,[0],[1.0])| 5.91|          0|             1|           675|            679|         3600.0|       0.0|
|6835508

In [41]:
for i in range(len(col_names)):
    df = df.withColumnRenamed(col_names[i], features[i])   

In [42]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- loan_status: integer (nullable = true)
 |-- loan_amnt: float (nullable = true)
 |-- int_rate: float (nullable = true)
 |-- installment: float (nullable = true)
 |-- term: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- sub_grade: integer (nullable = true)
 |-- emp_length: vector (nullable = true)
 |-- home_ownership: vector (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- verification_status: vector (nullable = true)
 |-- purpose: vector (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- fico_range_low: integer (nullable = true)
 |-- fico_range_high: integer (nullable = true)
 |-- total_rec_prncp: float (nullable = true)
 |-- recoveries: float (nullable = true)



In [43]:
df.count()

1266325

In [44]:
df.show(5)

+--------+-----------+---------+--------+-----------+----+-----+---------+--------------+--------------+----------+-------------------+--------------+-----+-----------+--------------+--------------+---------------+---------------+----------+
|      id|loan_status|loan_amnt|int_rate|installment|term|grade|sub_grade|    emp_length|home_ownership|annual_inc|verification_status|       purpose|  dti|delinq_2yrs|inq_last_6mths|fico_range_low|fico_range_high|total_rec_prncp|recoveries|
+--------+-----------+---------+--------+-----------+----+-----+---------+--------------+--------------+----------+-------------------+--------------+-----+-----------+--------------+--------------+---------------+---------------+----------+
|68407277|          1|   3600.0|   13.99|     123.03|   0|    2|       13|(10,[0],[1.0])| (3,[0],[1.0])|   55000.0|      (2,[1],[1.0])|(13,[0],[1.0])| 5.91|          0|             1|           675|            679|         3600.0|       0.0|
|68355089|          1|  24700.0|

In [45]:
df.sort(col("delinq_2yrs").desc()).show(5)

+---------+-----------+---------+--------+-----------+----+-----+---------+--------------+--------------+----------+-------------------+--------------+-----+-----------+--------------+--------------+---------------+---------------+----------+
|       id|loan_status|loan_amnt|int_rate|installment|term|grade|sub_grade|    emp_length|home_ownership|annual_inc|verification_status|       purpose|  dti|delinq_2yrs|inq_last_6mths|fico_range_low|fico_range_high|total_rec_prncp|recoveries|
+---------+-----------+---------+--------+-----------+----+-----+---------+--------------+--------------+----------+-------------------+--------------+-----+-----------+--------------+--------------+---------------+---------------+----------+
| 46305428|          1|  12500.0|   17.57|     314.51|   1|    3|       18|(10,[0],[1.0])| (3,[1],[1.0])|   25000.0|      (2,[1],[1.0])|(13,[3],[1.0])|32.69|         39|             1|           665|            669|        12500.0|       0.0|
| 65373706|          1|  150

In [46]:
df.sort("loan_amnt").show(5)

+------+-----------+---------+--------+-----------+----+-----+---------+--------------+--------------+----------+-------------------+--------------+-----+-----------+--------------+--------------+---------------+---------------+----------+
|    id|loan_status|loan_amnt|int_rate|installment|term|grade|sub_grade|    emp_length|home_ownership|annual_inc|verification_status|       purpose|  dti|delinq_2yrs|inq_last_6mths|fico_range_low|fico_range_high|total_rec_prncp|recoveries|
+------+-----------+---------+--------+-----------+----+-----+---------+--------------+--------------+----------+-------------------+--------------+-----+-----------+--------------+--------------+---------------+---------------+----------+
|242695|          1|    500.0|   10.71|      16.31|   0|    1|        9|(10,[2],[1.0])| (3,[0],[1.0])|   7904.04|      (2,[1],[1.0])|(13,[9],[1.0])| 3.04|          1|             2|           680|            684|          500.0|       0.0|
|164346|          1|    500.0|    8.07| 

Ok almost everything seems to be in order. I need one extra check to make sure the data is clean. Next I'm going to handle outliers, logical consistencies, and known ranges. 

In [47]:
df.filter(col('fico_range_low') > col('fico_range_high')).count()

0

In [48]:
df.createOrReplaceTempView('cleanDataset')

In [49]:
query = spark.sql('''
SELECT * FROM cleanDataset
WHERE (fico_range_low < 300 OR fico_range_low > 850)
                  OR (fico_range_high < 300 OR fico_range_high > 850)
''')
query.count()

0

In [50]:
query = spark.sql('''SELECT * FROM cleanDataset
                  WHERE loan_amnt < 0
''')
query.count()

0

I checked manually for negative values and max loan amount using the sort function for each column (that's numbered anyway). Now time to handle outline.

In [51]:
def printQuantiles(column):
    quant = df.approxQuantile(column, probabilities = [0.25, 0.75], relativeError = 0)
    IQR = quant[1] - quant[0]
    lower = quant[0] - 1.5 * IQR
    upper = quant[1] + 1.5 * IQR
    print(column)
    print(f'Lower : {lower}')
    print(f'Upper : {upper}')
    print(f'IQR : {IQR}')

In [52]:
#for feature in colsInts:
    #printQuantiles(feature)


In [53]:
#for feature in colsFloats:
    #printQuantiles(feature)

Ok so looking through all the information there is no one size fits all appraoch, you'd have to do deep analysis of each feature to figure out the best appraoch. Now, in general I'm going use a general approach of capping outliers to a certain value. I'm doing this because extreme values are legit but could be too influential during model training. 

In [54]:
capVals = {"loan_amnt" : 35000, "int_rate" : 30, "installment" : 1100,
           "annual_inc" : 250000, "dti" : 45, "delinq_2yrs" : 4, 
           "inq_last_6mths" : 6}

In [55]:
df.describe().show()

+-------+--------------------+------------------+-----------------+------------------+-----------------+-------------------+-----------------+------------------+-----------------+-----------------+-------------------+------------------+-----------------+------------------+------------------+------------------+
|summary|                  id|       loan_status|        loan_amnt|          int_rate|      installment|               term|            grade|         sub_grade|       annual_inc|              dti|        delinq_2yrs|    inq_last_6mths|   fico_range_low|   fico_range_high|   total_rec_prncp|        recoveries|
+-------+--------------------+------------------+-----------------+------------------+-----------------+-------------------+-----------------+------------------+-----------------+-----------------+-------------------+------------------+-----------------+------------------+------------------+------------------+
|  count|             1266325|           1266325|          12663

In [56]:
from pyspark.ml.functions import vector_to_array
for feature in encodedCols:
    df = df.withColumn(feature, vector_to_array(col(feature)))

In [57]:
spark_df = df

In [58]:
df.write.mode('overwrite').parquet("cleanLoans.parquet")

In [203]:
dataset = pd.read_parquet("cleanLoans.parquet")

In [60]:
dataset

,id,loan_status,loan_amnt,int_rate,installment,term,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs,inq_last_6mths,fico_range_low,fico_range_high,total_rec_prncp,recoveries
0,68407277,1,3600.0,13.990000,123.029999,0,2,13,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]",55000.0,"[0.0, 1.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5.910000,0,1,675,679,3600.000000,0.0
1,68355089,1,24700.0,11.990000,820.280029,0,2,10,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]",65000.0,"[0.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",16.059999,1,4,715,719,24700.000000,0.0
2,68341763,1,20000.0,10.780000,432.660004,1,1,8,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]",63000.0,"[0.0, 1.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.780000,0,0,695,699,20000.000000,0.0
3,68476807,1,10400.0,22.450001,289.910004,1,5,25,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]",104433.0,"[1.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",25.370001,1,3,695,699,10400.000000,0.0
4,68426831,1,11950.0,13.440000,405.179993,0,2,12,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0]",34000.0,"[1.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.200000,0,0,690,694,11950.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266320,89905081,1,18000.0,9.490000,377.950012,1,1,6,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0]",130000.0,"[0.0, 1.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",20.590000,0,1,735,739,18000.000000,0.0
1266321,88948836,1,29400.0,13.990000,683.940002,1,2,12,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]",180792.0,"[0.0, 1.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",22.030001,0,1,705,709,29400.000000,0.0
1266322,89996426,0,32000.0,14.490000,752.739990,1,2,13,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]",157000.0,"[1.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.340000,0,0,735,739,1876.469971,0.0
1266323,90006534,1,16000.0,12.790000,362.339996,1,2,10,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0]",150000.0,"[0.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",12.250000,0,0,665,669,16000.000000,0.0


In [61]:
def expandColumn(dataset, colName):
    columns = []
    for i in range(len(dataset[colName][0])):
        columns.append(f"{colName}_{i}")
    arr = pd.DataFrame(dataset[colName].tolist(), columns = columns)
    dataset = dataset.drop(columns = [colName])
    return pd.concat([dataset, arr], axis = 1)

In [230]:
df = dataset
for feature in encodedCols:
    df = expandColumn(df, feature)

In [231]:
dataset

,id,loan_status,loan_amnt,int_rate,installment,term,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs,inq_last_6mths,fico_range_low,fico_range_high,total_rec_prncp,recoveries
0,68407277,1,3600.0,13.990000,123.029999,0,2,13,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]",55000.0,"[0.0, 1.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5.910000,0,1,675,679,3600.000000,0.0
1,68355089,1,24700.0,11.990000,820.280029,0,2,10,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]",65000.0,"[0.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",16.059999,1,4,715,719,24700.000000,0.0
2,68341763,1,20000.0,10.780000,432.660004,1,1,8,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]",63000.0,"[0.0, 1.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.780000,0,0,695,699,20000.000000,0.0
3,68476807,1,10400.0,22.450001,289.910004,1,5,25,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]",104433.0,"[1.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",25.370001,1,3,695,699,10400.000000,0.0
4,68426831,1,11950.0,13.440000,405.179993,0,2,12,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0]",34000.0,"[1.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.200000,0,0,690,694,11950.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266320,89905081,1,18000.0,9.490000,377.950012,1,1,6,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0]",130000.0,"[0.0, 1.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",20.590000,0,1,735,739,18000.000000,0.0
1266321,88948836,1,29400.0,13.990000,683.940002,1,2,12,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]",180792.0,"[0.0, 1.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",22.030001,0,1,705,709,29400.000000,0.0
1266322,89996426,0,32000.0,14.490000,752.739990,1,2,13,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]",157000.0,"[1.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.340000,0,0,735,739,1876.469971,0.0
1266323,90006534,1,16000.0,12.790000,362.339996,1,2,10,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0]",150000.0,"[0.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",12.250000,0,0,665,669,16000.000000,0.0


In [63]:
trainDF = df.drop(axis = 1, columns = ['id', 'total_rec_prncp', 'grade', 'recoveries'])

In [64]:
for key, values in capVals.items():
    trainDF[key].clip(lower = 0, upper = values)
trainDF

,loan_status,loan_amnt,int_rate,installment,term,sub_grade,annual_inc,dti,delinq_2yrs,inq_last_6mths,...,purpose_3,purpose_4,purpose_5,purpose_6,purpose_7,purpose_8,purpose_9,purpose_10,purpose_11,purpose_12
0,1,3600.0,13.990000,123.029999,0,13,55000.0,5.910000,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,24700.0,11.990000,820.280029,0,10,65000.0,16.059999,1,4,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,20000.0,10.780000,432.660004,1,8,63000.0,10.780000,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,10400.0,22.450001,289.910004,1,25,104433.0,25.370001,1,3,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,11950.0,13.440000,405.179993,0,12,34000.0,10.200000,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266320,1,18000.0,9.490000,377.950012,1,6,130000.0,20.590000,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1266321,1,29400.0,13.990000,683.940002,1,12,180792.0,22.030001,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1266322,0,32000.0,14.490000,752.739990,1,13,157000.0,10.340000,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1266323,1,16000.0,12.790000,362.339996,1,10,150000.0,12.250000,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Model Building 

Logistic Regression with Train Test Split

In [65]:
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, roc_auc_score

In [66]:
X = trainDF.drop(columns = ['loan_status']).to_numpy()
Y = trainDF['loan_status'].to_numpy()

In [67]:
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.2,  
                                                    random_state = 42, stratify = Y)

In [68]:
scaler = StandardScaler()
xTrainS = scaler.fit_transform(xTrain)
xTestS = scaler.transform(xTest)

In [69]:
model = LogisticRegression(max_iter = 1000, class_weight = 'balanced')
model.fit(xTrainS, yTrain)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [70]:
yPred = model.predict(xTestS)
yProb = model.predict_proba(xTestS)[:, 0]

In [71]:
print(classification_report(yTest, yPred))

              precision    recall  f1-score   support

           0       0.32      0.64      0.42     49475
           1       0.88      0.67      0.76    203790

    accuracy                           0.66    253265
   macro avg       0.60      0.65      0.59    253265
weighted avg       0.77      0.66      0.69    253265



In [72]:
roc_auc1 = roc_auc_score(yTest,
                         model.predict_proba(xTestS)[:, 1])           # class 1
roc_auc0 = roc_auc_score(yTest,
                         model.predict_proba(xTestS)[:, 0])                                 # class 0

print(f"AUC (Fully-Paid as positive):  {roc_auc1:.3f}")
print(f"AUC (Charged-Off as positive): {roc_auc0:.3f}")

AUC (Fully-Paid as positive):  0.710
AUC (Charged-Off as positive): 0.290


With Cross Validation Now (5 Fold)

In [73]:
pipeline = make_pipeline(StandardScaler(), 
                         LogisticRegression(max_iter = 1000, class_weight = 'balanced'))

In [74]:
cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

In [75]:
auc = cross_val_score(pipeline, X, Y, cv = cv, scoring = 'roc_auc', n_jobs = -1)

In [76]:
print(f"5-fold AUC:  mean={auc.mean():.3f},  std={auc.std():.3f}")

5-fold AUC:  mean=0.710,  std=0.002


Result Analysis

In [77]:
probs = pd.DataFrame(yProb, columns = ["Probability of Default"])
probs = probs * 100
probs

,Probability of Default
0,13.522749
1,54.930830
2,52.413067
3,27.426711
4,35.073038
...,...
253260,53.861373
253261,32.347839
253262,50.143497
253263,26.878072


In [78]:
# Fully Paid = 1 and Charged Off = 0
pred = pd.DataFrame(yPred, columns = ['Prediction'])
pred

,Prediction
0,1
1,0
2,0
3,1
4,1
...,...
253260,0
253261,1
253262,0
253263,1


In [79]:
# Fully Paid = 1 and Charged Off = 0
test = pd.DataFrame(yTest, columns = ['Real Values'])
test

,Real Values
0,1
1,1
2,1
3,1
4,0
...,...
253260,1
253261,0
253262,1
253263,0


In [80]:
results = pd.concat([probs, pred, test], axis = 1)
results

,Probability of Default,Prediction,Real Values
0,13.522749,1,1
1,54.930830,0,1
2,52.413067,0,1
3,27.426711,1,1
4,35.073038,1,0
...,...,...,...
253260,53.861373,0,1
253261,32.347839,1,0
253262,50.143497,0,1
253263,26.878072,1,0


In [81]:
from sqlalchemy import create_engine 
from sqlalchemy.exc import OperationalError
import os
from dotenv import load_dotenv
load_dotenv()
server   = os.getenv("SQL_SERVER")
database = os.getenv("SQL_DATABASE")
username = os.getenv("SQL_USERNAME")
password = os.getenv("SQL_PASSWORD")

In [82]:
conn_url = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+18+for+SQL+Server'
engine = create_engine(conn_url)
try:
    results.to_sql(name="LoanPredictions", con=engine, if_exists="replace", index=False)
except OperationalError as e:
    print('Try Again')
    results.to_sql(name="LoanPredictions", con=engine, if_exists="replace", index=False)

In [83]:
resultsPT = results.groupby(['Prediction', 'Real Values']).count()
resultsPT = resultsPT.rename(columns={'Probability of Default': 'Count'})
resultsPT

Count
Prediction Real Values        
0          0             31468
           1             67746
1          0             18007
           1            136044

## Basel III Based Calculations

In [89]:
# q = df[df['loan_status_0'] == 1].sort_values(by = ['EAD', 'id'], ascending = True)[['id', 'loan_status_0', 'LGD','recoveries', 'loan_amnt', 'total_rec_prncp', 'EAD']]
# q = q[(q['EAD'] != 0) | (q['LGD'] != 0)]
# q = q[q['EAD'] != (q['loan_amnt'] - q['total_rec_prncp'])]
# q

In [233]:
xAll = df.drop(columns = ['loan_status', 'id', 'total_rec_prncp', 'grade', 'recoveries'])
xAllS = scaler.transform(xAll)
df['PD'] = model.predict_proba(xAllS)[:, 0]

c:\Users\ty725\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [234]:
df['EAD'] = df['loan_amnt'] - df['total_rec_prncp']
df['EAD'] = np.where(
    (df['loan_status'] == 1) & (df['EAD'] < 2),
    0,
    df['EAD']
    )

In [235]:
df['LGD'] = 1 - (df['recoveries'] / df['EAD'])
df['LGD'] = df['LGD'].clip(lower = 0, upper = 1)
df['LGD'] = df['LGD'].fillna(0)

To ensure the integrity of expected loss (ECL), economic capital (EC), and risk-weighted asset (RWA) estimates, three anomaly cases were identified and addressed. (1) Fully Paid loans with small non-zero EAD values due to rounding were retained if EAD < $2 or repayment exceeded 99.9%, and treated as effectively settled. (2) A set of 1,543 Fully Paid loans with EAD ≥ $25 and repayment ratios below 95% were flagged as likely operational anomalies or mislabeling cases, and excluded from EL, EC, and RWA calculations. (3) Lastly, 18 loans marked as Charged Off or Defaulted but with 100% principal repayment were identified as post-default recoveries or labeling issues and also excluded from capital modeling. These cases were flagged for transparency but excluded from financial risk measures to avoid distortion of portfolio-level capital requirements.

In [236]:
def calcs(df):
    from scipy.stats import norm
    
    calc = df.copy()

    # Calculate z-scores for PD
    gPD = norm.ppf(calc['PD'])
    g999 = norm.ppf(0.999)

    # Calculate repayment ratio
    calc['repayment_ratio'] = calc['total_rec_prncp'] / calc['loan_amnt']

    # --- Flag Anomalies ---

    # Case 2: Fully Paid loans with large EAD and repayment ratio < 0.95
    calc['case2_flag'] = (
        (calc['loan_status'] == 'Fully Paid') &
        (calc['EAD'] >= 25) &
        (calc['repayment_ratio'] < 0.95)
    )

    # Case 3: Defaulted loans that repaid full principal
    calc['case3_flag'] = (
        (calc['loan_status'].isin(['Default', 'Charged Off'])) &
        (calc['repayment_ratio'] == 1)
    )

    # Global exclusion flag
    calc['exclude_flag'] = calc['case2_flag'] | calc['case3_flag']

    # Expected Loss (EL or ECL)
    calc['EL'] = np.where(
        calc['exclude_flag'],
        0,
        calc['LGD'] * calc['PD'] * calc['EAD']
    )

    # Asset correlation (R)
    calc['R'] = 0.03 * (1 - np.exp(-35 * calc['PD'])) + 0.16 * np.exp(-35 * calc['PD'])

    # Capital requirement (K)
    capital_term = (
        (1 / np.sqrt(1 - calc['R'])) * gPD +
        np.sqrt(calc['R'] / (1 - calc['R'])) * g999
    )

    calc['K'] = np.where(
        calc['exclude_flag'],
        0,
        calc['LGD'] * norm.cdf(capital_term) - calc['PD'] * calc['LGD']
    )

    # Economic Capital (EC) and RWA
    calc['EC'] = calc['K'] * calc['EAD']
    calc['RWA'] = 12.5 * calc['EC']

    return calc


Okay all of the important calculations were made according to the Basel III approach (PD, LGD, EAD, EL). Time to put it in Spark so that I can join it with the original dataset by joining the IDs and put in in Azure (both the logistic regression model and the full clean dataset with the added calculations).

We also have to calculate for Risk Weighted Assets and Economic Capital

In [237]:
query = spark.sql('''
SELECT l.id, l.loan_status, l.loan_amnt, l.int_rate, l.installment, l.grade, l.sub_grade, 
l.emp_length, l.home_ownership, l.annual_inc, l.verification_status, 
l.purpose, l.dti, l.delinq_2yrs, l.inq_last_6mths, l.fico_range_low, 
l.fico_range_high, l.total_rec_prncp, l.recoveries, l.term
FROM loans as l
JOIN cleanDataset AS c ON l.id = c.id
''')

In [239]:
query.write.mode('overwrite').parquet('CreditRiskStats')

In [240]:
stats = pd.read_parquet("CreditRiskStats")

In [241]:
colNums = '''
loan_amnt, int_rate, installment, annual_inc, dti, delinq_2yrs, inq_last_6mths, 
fico_range_low, fico_range_high, total_rec_prncp
'''
colNums = cleanStrings(colNums)

for c in colNums:
    stats[c] = pd.to_numeric(stats[c])

In [242]:
stats['loan_status'] = stats['loan_status'].replace('Charged Off', 'Default')

In [243]:
stats = pd.merge(stats, df[['id','PD', 'LGD', 'EAD']], on = ['id'], how = 'inner')
stats

,id,loan_status,loan_amnt,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,...,delinq_2yrs,inq_last_6mths,fico_range_low,fico_range_high,total_rec_prncp,recoveries,term,PD,LGD,EAD
0,100002366,Fully Paid,25000.0,13.49,848.27,C,C2,2 years,MORTGAGE,60000.0,...,0.0,0.0,710.0,714.0,25000.00,0.0,36 months,0.374848,0.00000,0.000000
1,100004206,Fully Paid,25000.0,12.74,839.23,C,C1,10+ years,OWN,130000.0,...,1.0,1.0,685.0,689.0,25000.00,0.0,36 months,0.460643,0.00000,0.000000
2,100005797,Fully Paid,12000.0,15.99,421.83,C,C5,10+ years,RENT,58750.0,...,0.0,2.0,665.0,669.0,12000.00,0.0,36 months,0.562959,0.00000,0.000000
3,100006378,Fully Paid,20000.0,7.99,626.64,A,A5,5 years,MORTGAGE,85000.0,...,0.0,1.0,700.0,704.0,20000.00,0.0,36 months,0.326667,0.00000,0.000000
4,1000067,Fully Paid,15000.0,16.77,370.94,D,D2,< 1 year,RENT,72000.0,...,0.0,0.0,705.0,709.0,15000.00,0.0,60 months,0.773598,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266320,99994464,Default,35000.0,13.99,1196.05,C,C3,9 years,OWN,125000.0,...,2.0,1.0,670.0,674.0,3204.77,4200.47,36 months,0.513288,0.86789,31795.230469
1266321,99995251,Fully Paid,10000.0,11.49,329.72,B,B5,10+ years,RENT,60000.0,...,0.0,0.0,665.0,669.0,10000.00,0.0,36 months,0.441274,0.00000,0.000000
1266322,99996794,Fully Paid,25000.0,16.99,891.20,D,D1,3 years,MORTGAGE,125000.0,...,0.0,1.0,675.0,679.0,25000.00,0.0,36 months,0.554658,0.00000,0.000000
1266323,99997099,Fully Paid,6000.0,11.44,197.69,B,B4,< 1 year,OWN,17000.0,...,0.0,0.0,700.0,704.0,6000.00,0.0,36 months,0.415067,0.00000,0.000000


In [245]:
filt = stats[stats['loan_status'] == 'Fully Paid']
count = filt[filt['EAD'] != 0].sort_values(by = ['EAD', 'id'], ascending = False)
count = count[['id', 'loan_amnt', 'total_rec_prncp', 'recoveries', 'PD', 'LGD', 'EAD']]
count

,id,loan_amnt,total_rec_prncp,recoveries,PD,LGD,EAD
1015887,1067125,30000.0,8850.0,0.0,0.854711,1.0,21150.0
343497,394329,25000.0,3975.0,0.0,0.635472,1.0,21025.0
508435,1031765,25000.0,4400.0,0.0,0.513069,1.0,20600.0
32756,312516,25000.0,5200.0,0.0,0.457610,1.0,19800.0
670084,613815,24000.0,4500.0,0.0,0.168115,1.0,19500.0
...,...,...,...,...,...,...,...
457701,5335336,27500.0,27475.0,0.0,0.845305,1.0,25.0
1166043,519527,25000.0,24975.0,0.0,0.628345,1.0,25.0
558335,519199,12350.0,12325.0,0.0,0.537460,1.0,25.0
344855,405031,4750.0,4725.0,0.0,0.460689,1.0,25.0


In [246]:
count['Missing'] = count['total_rec_prncp'] / count['loan_amnt']
count[count['Missing'] >= 0.99]

,id,loan_amnt,total_rec_prncp,recoveries,PD,LGD,EAD,Missing
558842,526511,23000.0,22825.0,0.0,0.689570,1.0,175.0,0.992391
1205853,884853,20000.0,19850.0,0.0,0.429408,1.0,150.0,0.992500
1066347,548406,25000.0,24875.0,0.0,0.776295,1.0,125.0,0.995000
1243667,5649239,33425.0,33325.0,0.0,0.700799,1.0,100.0,0.997008
1165696,5177165,32000.0,31900.0,0.0,0.852806,1.0,100.0,0.996875
968982,578726,23500.0,23450.0,0.0,0.681443,1.0,50.0,0.997872
813375,1068416,7250.0,7200.0,0.0,0.870279,1.0,50.0,0.993103
74036,7004907,2875.0,2850.0,0.0,0.437562,1.0,25.0,0.991304
469491,628545,8000.0,7975.0,0.0,0.175687,1.0,25.0,0.996875
356437,5345475,25000.0,24975.0,0.0,0.858273,1.0,25.0,0.999000


In [247]:
df_anom = stats[(stats['loan_status'] == 'Fully Paid') & (stats['EAD'] >= 25)]
df_anom['delta_ratio'] = df_anom['total_rec_prncp'] / df_anom['loan_amnt']
df_anom['EAD_bucket'] = pd.cut(df_anom['EAD'], bins=[25, 50, 100, 500, 1000, 5000, np.inf])

df_anom.groupby('EAD_bucket')['delta_ratio'].describe()

C:\Users\ty725\AppData\Local\Temp\ipykernel_15960\248939168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anom['delta_ratio'] = df_anom['total_rec_prncp'] / df_anom['loan_amnt']
C:\Users\ty725\AppData\Local\Temp\ipykernel_15960\248939168.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anom['EAD_bucket'] = pd.cut(df_anom['EAD'], bins=[25, 50, 100, 500, 1000, 5000, np.inf])
C:\Users\ty725\AppData\Local\Temp\ipykernel_15960\248939168.py:5: FutureWarning: The default of observed=False is depreca

,count,mean,std,min,25%,50%,75%,max
EAD_bucket,,,,,,,,
"(25.0, 50.0]",2.0,0.995488,0.003372,0.993103,0.994296,0.995488,0.996680,0.997872
"(50.0, 100.0]",3.0,0.992406,0.007857,0.983333,0.990104,0.996875,0.996942,0.997008
"(100.0, 500.0]",30.0,0.949473,0.049223,0.783333,0.925000,0.969792,0.984418,0.995000
"(500.0, 1000.0]",47.0,0.890124,0.102676,0.525000,0.865000,0.927083,0.956472,0.978571
"(1000.0, 5000.0]",609.0,0.734927,0.101426,0.343750,0.666667,0.735417,0.807143,0.952083
"(5000.0, inf]",844.0,0.637807,0.097236,0.101250,0.620000,0.638194,0.678083,0.852857


In [248]:
defaults = stats[stats['loan_status'] == 'Default']
defaults1 = defaults[defaults['EAD'] == 0]
defaults1[['id', 'recoveries', 'loan_amnt', 'total_rec_prncp', 'LGD', 'EAD', 'PD']]

,id,recoveries,loan_amnt,total_rec_prncp,LGD,EAD,PD
165416,63155120,0.0,6000.0,6000.0,0.0,0.0,0.318537
197876,93832361,0.0,9000.0,9000.0,0.0,0.0,0.329103
292276,85751192,0.0,6925.0,6925.0,0.0,0.0,0.352637
376856,68996838,0.0,7800.0,7800.0,0.0,0.0,0.196495
611628,107145470,0.0,29000.0,29000.0,0.0,0.0,0.596169
618411,119413934,0.0,12000.0,12000.0,0.0,0.0,0.595880
620831,124444114,0.0,15000.0,15000.0,0.0,0.0,0.665444
621463,125089586,0.0,9600.0,9600.0,0.0,0.0,0.197739
684214,71744763,37.67,14500.0,14500.0,0.0,0.0,0.400754
781653,68884885,0.0,25000.0,25000.0,0.0,0.0,0.759722


In [249]:
defaults[defaults['EAD'] > 0].shape[0] / stats.shape[0]

0.1953345310248159

In [263]:
calcFeats = ['PD', 'LGD', 'EAD', 'EL', 'EC', 'RWA']
for feat in calcFeats:
    features.append(feat)


In [265]:
stats = calcs(stats)[features]

In [266]:
stats

,id,loan_status,loan_amnt,int_rate,installment,term,grade,sub_grade,emp_length,home_ownership,...,fico_range_low,fico_range_high,total_rec_prncp,recoveries,PD,LGD,EAD,EL,EC,RWA
0,100002366,Fully Paid,25000.0,13.49,848.27,36 months,C,C2,2 years,MORTGAGE,...,710.0,714.0,25000.00,0.0,0.374848,0.00000,0.000000,0.000000,0.000000,0.000000
1,100004206,Fully Paid,25000.0,12.74,839.23,36 months,C,C1,10+ years,OWN,...,685.0,689.0,25000.00,0.0,0.460643,0.00000,0.000000,0.000000,0.000000,0.000000
2,100005797,Fully Paid,12000.0,15.99,421.83,36 months,C,C5,10+ years,RENT,...,665.0,669.0,12000.00,0.0,0.562959,0.00000,0.000000,0.000000,0.000000,0.000000
3,100006378,Fully Paid,20000.0,7.99,626.64,36 months,A,A5,5 years,MORTGAGE,...,700.0,704.0,20000.00,0.0,0.326667,0.00000,0.000000,0.000000,0.000000,0.000000
4,1000067,Fully Paid,15000.0,16.77,370.94,60 months,D,D2,< 1 year,RENT,...,705.0,709.0,15000.00,0.0,0.773598,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266320,99994464,Default,35000.0,13.99,1196.05,36 months,C,C3,9 years,OWN,...,670.0,674.0,3204.77,4200.47,0.513288,0.86789,31795.230469,14164.048548,5652.456593,70655.707412
1266321,99995251,Fully Paid,10000.0,11.49,329.72,36 months,B,B5,10+ years,RENT,...,665.0,669.0,10000.00,0.0,0.441274,0.00000,0.000000,0.000000,0.000000,0.000000
1266322,99996794,Fully Paid,25000.0,16.99,891.20,36 months,D,D1,3 years,MORTGAGE,...,675.0,679.0,25000.00,0.0,0.554658,0.00000,0.000000,0.000000,0.000000,0.000000
1266323,99997099,Fully Paid,6000.0,11.44,197.69,36 months,B,B4,< 1 year,OWN,...,700.0,704.0,6000.00,0.0,0.415067,0.00000,0.000000,0.000000,0.000000,0.000000


In [268]:
engine = create_engine(conn_url)
try:
    stats.to_sql(name="CreditRisk", con=engine, if_exists="replace", index=False)
except OperationalError as e:
    print('Try again')
    stats.to_sql(name="CreditRisk", con=engine, if_exists="replace", index=False)

## Playing Around with Threshold

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

# y_test  -> true labels   (0 = Charged-Off, 1 = Fully-Paid)
# y_prob  -> model-predicted probability of class 1 (Fully-Paid)
# If you stored P(class 0) instead, use 1-y_prob

precision, recall, thresholds = precision_recall_curve(yTest, 1 - yProb, pos_label=0)

plt.figure(figsize=(6,4))
plt.plot(thresholds, precision[:-1], label='Precision (class 0)')
plt.plot(thresholds, recall[:-1],    label='Recall (class 0)')
plt.xlabel("Decision threshold for class 0")
plt.ylabel("Score")
plt.title("Precision & Recall vs Threshold (Charged-Off class)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
prec, rec, thr = precision_recall_curve(yTest, 1 - yProb, pos_label=0)
f1 = 2*(prec[:-1]*rec[:-1])/(prec[:-1]+rec[:-1])
best_thr = thr[f1.argmax()]
print("Best F1 at", best_thr)
y_pred_opt = ((1 - yProb) >= best_thr).astype(int)
print(classification_report(yTest, y_pred_opt))

In [ ]:
thr = 0.55        # example from the plot
y_pred_adj = ((1 - yProb) >= thr).astype(int)
y_pred_adj = pd.DataFrame(y_pred_adj, columns = ['Adjusted'])
show = pd.concat([y_pred_adj, test], axis = 1)
show

In [ ]:
print(classification_report(yTest, y_pred_adj))

## Export Model to Azure Machine Learning


In [ ]:
import joblib

In [ ]:
joblib.dump(model, "logistic_model.pkl")